In [ ]:
# Another option for date deployed chart

levels = np.tile([6, -4,  3, -3, 2, -8, 7, -5, 6,  -5, 1, -1],
                 int(np.ceil(len(dates)/12)))[:len(dates)]

fig, ax = plt.subplots(figsize=(10, 3))
ax.set(title="Sensor Startup Date")

markerline, stemline, baseline = ax.stem(dates, levels,
                                         linefmt="k-", basefmt="k-",
                                         use_line_collection=True)

plt.setp(markerline, mec="k", mfc="w", zorder=3)
markerline.set_ydata(np.zeros(len(dates)))

vert = np.array(['top', 'bottom'])[(levels > 0).astype(int)]
for d, l, r, va in zip(dates, levels, device_ids, vert):
    ax.annotate(r, xy=(d, l), xytext=(-3, np.sign(l)*3),
                textcoords="offset points", va=va, ha="right")

# Format X axis with 4 month intervals
ax.get_xaxis().set_major_locator(mdates.DayLocator(interval=5))
ax.get_xaxis().set_major_formatter(mdates.DateFormatter("%d %b %Y"))
plt.setp(ax.get_xticklabels(), rotation=30, ha="right")

# Remove Y axis and spines
ax.get_yaxis().set_visible(False)
for spine in ["left", "top", "right"]:
    ax.spines[spine].set_visible(False)

ax.margins(y=0.5)
plt.show()

In [ ]:
# Plot sensor locations on state and county maps
# TO DO - FIX LABELS
import altair as alt
from vega_datasets import data
alt.renderers.enable('notebook')

# Load the data as a dict object
us_10m  = data.us_10m()
# Select the geometries under states under objects, filter for California (id = 6)
us_10m['objects']['states']['geometries']=[item for item in us_10m['objects'] \
      ['states']['geometries'] if item['id'] in [6]]
# Make California topojson data
state = alt.Data(
    values=us_10m, 
    format=alt.TopoDataFormat(feature='states',type='topojson'))
# Make California background
background = alt.Chart(state).mark_geoshape(
    fill='lightgray',
    stroke='white'
).properties(
    width=500,
    height=300
).project('mercator')

# Make SB County topojson data
SB_County = alt.Data(url = 'https://raw.githubusercontent.com/johan/world.geo.json/master/countries/USA/CA/Santa%20Barbara.geo.json', format=alt.DataFormat(property='features',type='json'))
# SB County as background
county = alt.Chart(SB_County).mark_geoshape(
    fill='lightblue',
    stroke='white'
).properties(
    width=500,
    height=300
).project('mercator')

# Plot sensor locations
points = alt.Chart(df_location).mark_point(
    size=1,
    color='black'
).encode(
    longitude='lon:Q',
    latitude='lat:Q'
)

# Label sensor names
text = points.mark_text(
    align='left',
    baseline='bottom',
    dx=9,
    dy=9,
    fontSize=9,
    tooltip={"content": "encoding"}
).encode(
    text='name:O'
)

ca = background + county + points 
sb = county + points + text
#alt.vconcat(ca, sb)
ca | sb

In [ ]:
def progress(count, total, status=''):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('[%s] %s%s ...%s\r' % (bar, percents, '%', status))
    sys.stdout.flush()